In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=ff8bf05170785e3bf002d759faf8271704fa89f1b3f1428c8009f02317090d6f
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
! pip install matplotlib
! pip install seaborn

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName("DecisionTreeRegression").getOrCreate()

df = spark.read.format("csv").option("header", True).load("fb_live_thailand.csv")

df = df.select(df.num_reactions.cast("Double"), df.num_loves.cast("Double"))

indexer_reactions = StringIndexer(inputCol="num_reactions", outputCol="num_reactions_ind")
indexer_loves = StringIndexer(inputCol="num_loves", outputCol="num_loves_ind")

encoder_reactions = OneHotEncoder(inputCol="num_reactions_ind", outputCol="reactions_encoded")
encoder_loves = OneHotEncoder(inputCol="num_loves_ind", outputCol="loves_encoded")

vec_assembler = VectorAssembler(inputCols=["reactions_encoded", "loves_encoded"], outputCol="features")

pipeline = Pipeline(stages=[indexer_reactions, indexer_loves, encoder_reactions, encoder_loves, vec_assembler])

pipeline_model = pipeline.fit(df)

df_transformed = pipeline_model.transform(df)

train_df, test_df = df_transformed.randomSplit([0.8, 0.2], seed=1234)

dt_regressor = DecisionTreeRegressor(featuresCol="features", labelCol="num_loves_ind")

dt_model = dt_regressor.fit(train_df)

predictions = dt_model.transform(test_df)

evaluator = RegressionEvaluator(labelCol="num_loves_ind", predictionCol="prediction")

r2 = evaluator.setMetricName("r2").evaluate(predictions)
print(f"R2: {r2}")


R2: 0.42418664164077347


In [ ]:
# ปรับให้ดี ค่า ใกล้ 1

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

# สร้าง SparkSession
spark = SparkSession.builder.appName("DecisionTreeRegression").getOrCreate()

# โหลดชุดข้อมูลจากไฟล์ CSV
df = spark.read.format("csv").option("header", True).load("fb_live_thailand.csv")

# แปลงคอลัมน์ num_reactions และ num_loves เป็นชนิดข้อมูล Double
df = df.select(df.num_reactions.cast("Double"), df.num_loves.cast("Double"))

# ลบแถวที่มีค่า null
df = df.na.drop()

# ใช้ StringIndexer เพื่อแปลงคอลัมน์ num_reactions และ num_loves เป็นค่าดัชนี
indexer_reactions = StringIndexer(inputCol="num_reactions", outputCol="num_reactions_ind")
indexer_loves = StringIndexer(inputCol="num_loves", outputCol="num_loves_ind")

# ใช้ OneHotEncoder เพื่อแปลงค่าดัชนีเป็นเวกเตอร์ที่เข้ารหัสแบบ One-Hot
encoder_reactions = OneHotEncoder(inputCols=["num_reactions_ind"], outputCols=["reactions_encoded"])
encoder_loves = OneHotEncoder(inputCols=["num_loves_ind"], outputCols=["loves_encoded"])

# ใช้ VectorAssembler เพื่อรวมฟีเจอร์เข้าด้วยกันเป็นเวกเตอร์เดียว
vec_assembler = VectorAssembler(inputCols=["reactions_encoded", "loves_encoded"], outputCol="features")

# สร้าง Pipeline ที่ประกอบด้วยขั้นตอนต่าง ๆ
pipeline = Pipeline(stages=[indexer_reactions, indexer_loves, encoder_reactions, encoder_loves, vec_assembler])

# ฝึก Pipeline model โดยใช้ชุดข้อมูล
pipeline_model = pipeline.fit(df)

# ทำการแปลงข้อมูลด้วย Pipeline model
df_transformed = pipeline_model.transform(df)

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
train_df, test_df = df_transformed.randomSplit([0.8, 0.2], seed=1234)

# สร้างโมเดล Decision Tree Regressor พร้อมตั้งค่า hyperparameters
dt_regressor = DecisionTreeRegressor(featuresCol="features", labelCol="num_loves_ind", maxDepth=30, minInstancesPerNode=2) # tune Model  maxDepth=30, minInstancesPerNode=2 ปรับตรงนี้

# ฝึกโมเดลด้วยชุดฝึก
dt_model = dt_regressor.fit(train_df)

# ทำการทำนายด้วยโมเดลที่ฝึกมา
predictions = dt_model.transform(test_df)

# สร้างตัวประเมินผลเพื่อประเมินโมเดล
evaluator = RegressionEvaluator(labelCol="num_loves_ind", predictionCol="prediction")

# คำนวณคะแนน R2
r2 = evaluator.setMetricName("r2").evaluate(predictions)
print(f"R2: {r2}")


R2: 0.7039013416577182
